In [20]:
import tensorflow as tf
import numpy as np
import matplotlib as plt
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import scipy
from PIL import Image

In [24]:

trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory= r"C:\Users\gonza\Downloads\dataset", target_size=(400,400))
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory=r"C:\Users\gonza\Downloads\dataset", target_size=(50,50))

Found 49460 images belonging to 2 classes.
Found 49460 images belonging to 2 classes.


In [26]:
#16 boxes de 50x50
tf.image.crop_and_resize(traindata,16,[50,50])
#tf.image.crop_to_bounding_box(testdata.classes[0]imag, 0, 0, 2, 2)
#tf.image.crop_to_bounding_box(testdata)

TypeError: Missing required positional argument

https://builtin.com/machine-learning/vgg16

https://towardsdatascience.com/transfer-learning-with-vgg16-and-keras-50ea161580b4